# Bae-s

In [75]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB, MultinomialNB
import scipy.stats as st
from matplotlib import pyplot as plt

### Problem 1

In [76]:
def Gaussian_NB(features, means, variances, prior):
    m, n = features.shape
    k = len(prior)
    stds = np.sqrt(variances)
    logprobs = np.zeros((m,k))
    logprobs += np.log(prior)
    for i in xrange(m):
        for j in xrange(k):
            for l in xrange(n):
                logprobs[i,j] += st.norm.logpdf(features[i,l], loc=means[l,j], scale=stds[l,j])
    return logprobs.argmax(axis=1)

Contains parts of Problem 1 and 2 below

### Problem 2

In [107]:
def seeds():
    data = pd.read_csv("seeds_dataset.txt",  delim_whitespace=True, names=
            "Area, Perimeter, Compactness, Length, Width, Asymmetry Coefficient, Groove Length, Class".split(", "))
    
    #test and train data
    test = data.loc[np.random.choice(data.index,40, replace=False)]
    train = data.loc[[i for i in set.difference(set(data.index), set(test.index))]]
    
    #calculate means and variances
    means = np.zeros((7,3))
    variances = np.zeros((7,3))
    for i in xrange(3):
        means[:,i] = np.array(train[train["Class"]==i+1].mean())[:-1]
        variances[:,i] = np.array(train[train["Class"]==i+1].var())[:-1]
        
    prior = np.ones(3)/3.
    labels = Gaussian_NB(np.array(test)[:,:-1], means, variances, prior) + 1
    truth = np.array(test)[:,-1]
    
    print "Accuracy:", (labels==truth).sum()/float(len(labels))
    nb = GaussianNB()
    nb.fit(np.array(train)[:,:-1], np.array(train)[:,-1])
    labels2 = nb.predict(np.array(test)[:,:-1])
    
    #check to see if same as sklearn
    if np.allclose(labels, labels2) == True:
        print "The SKLearn library got the same thing."
    else:
        print "Didn't get the same thing"
seeds()

Accuracy: 0.95
The SKLearn library got the same thing.


### Problem 3

In [90]:
class naiveBayes(object):
    """
    This class performs naive bayes classification for word-count document features.
    """
    def __init__(self):
        """
        Initialize a naive Bayes classifier.
        """
        self.n_samples, self.n_features, self.class_probs, self.n_classes, self.word_class_probs = [None]*5

    def fit(self,X,Y):
        """
        Fit the parameters according to the labeled training data (X,Y).
        Parameters
        ----------
        X : numpy arrary of shape [n_samples, n_features].
            Each row is essentially the "word-count" vector for one of the "documents".
        Y : numpy array of shape [n_samples].
            Gives the class label for each instance of training data. Assume class labels
            are {0,1,...,k-1} where k is the number of classes.
        """
        self.n_samples, self.n_features = X.shape
        self.class_probs = np.array([(Y==i).sum() for i in set(Y)])/float(self.n_samples)
        self.n_classes = len(self.class_probs)
        self.word_class_probs = np.empty([self.n_classes,self.n_features])
        for c in xrange(self.n_classes):
            self.word_class_probs[c,:] = (X[Y==c].sum(axis=0)+1).T
            self.word_class_probs[c,:] /= self.word_class_probs[c,:].sum()

    def predict(self, X):
        """
        Predict the class labels of a set of test data.
        Parameters
        ----------
        X : numpy array of shape [n_samples, n_features]
        Returns
        -------
        Y : numpy array of shape [n_samples].
            Gives the classification of each row in X
        """
        return np.argmax(np.log(self.class_probs) + X.dot(np.log(self.word_class_probs).T), axis=1)

### Problem 4

In [92]:
def spam():
    dat = np.loadtxt("SpamFeatures.txt")
    labs = np.loadtxt("SpamLabels.txt")
    nb = naiveBayes()
    
    test_rows = np.random.choice(np.arange(len(labs)),500, replace=False)
    train_rows = np.array([i for i in xrange(len(labs)) if i not in test_rows])
    nb.fit(dat[train_rows], labs[train_rows])
    new_labs = nb.predict(dat[test_rows])
    
    print (new_labs==labs[test_rows]).sum()/float(len(new_labs))
    mnb = MultinomialNB()
    mnb.fit(dat[train_rows], labs[train_rows])
    new_labs2 = mnb.predict(dat[test_rows])
    
    print (new_labs2 == labs[test_rows]).sum()/float(len(new_labs))
    
    if np.allclose((new_labs==labs[test_rows]).sum()/float(len(new_labs)),(new_labs2 == labs[test_rows]).sum()/float(len(new_labs))) == True:
        print "They are the same."
    
    
spam()

0.95
0.95
They are the same.
